In [1]:
import networkx as nx
import pandas as pd
import os
import random
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict 
from dgl.nn import SAGEConv
import scipy.sparse as sp
import json
import seaborn as sns
import matplotlib.pyplot as plt

import dgl
from dgl.data.utils import save_graphs
from dgl import load_graphs

from dgl.nn import SAGEConv
import dgl.data

Using backend: pytorch


# Recommender Pipeline

The recommender will load in the trained model.

Select all songs from a playlist, get ranked song recommendations for each

Each set of ranked song recommendations will be an edgelist of (source, destination) where destination is the recommended song.

Get the highest rank from each song each set of recommendations and rank those

Return those songs

In [ ]:
from collections import defaultdict

In [ ]:
fname = (os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.0-999.json'))
'''
Get int ids for each track in playlist
'''
def pull_tracks(fname, label_to_int):
    with open(fname) as f:
        data = json.load(f)
        item = np.random.choice(data['playlists']) # Get a random playlist to recommend for
    
    # Get int values from track labels
    int_vals = []
    for track in item['tracks']:
        code = track['track_uri'].replace('spotify:track:', '')
        int_vals.append(label_to_int[code])
        
    return int_vals

In [ ]:
'''
Create recommendations for each song in the playlist.
'''
def recommend(int_vals, model):
    recs = defaultdict(list) # dictionary of recommendations in form {'id': '(rec1, rank), '(rec2, rank)}
    for i in int_vals:
        preds = model.predict(i)
        for j in preds: # keep only predictions not already in playlist
            if j['id'] not in int_vals:
                recs[i].append((j['id'], j['rank']))
                
    return recs

In [ ]:
'''
Rank all the recommendations, taking the highest recommended song
'''
def rank(recs):
    final = []
    for i in recs.keys():
        recs[i].sort(key = lambda x:x[1], reverse=True)
        # Get highest rated recommendation for each song
        final.append(recs[i][0])
        
    # Sort by best ranks
    final.sort(key = lambda x:x[1], reverse=True)
    ranked = list(zip(*final))[0]
    return ranked

In [ ]:
'''
Decode to song labels
'''
def decode(ranked, int_to_label):
    decoded = []
    for i in ranked:
        decoded.append(int_to_label[i])
    return decoded

In [2]:
#%%time
#G = bf.load_data("data/", "Spotify Playlist", 140, 300, 1000, False)

In [3]:
os.path.exists(os.path.join(os.path.expanduser('~'), 'test_save.bin'))

True

In [4]:
%%time
dglG = load_graphs(os.path.join(os.path.expanduser('~'), 'test_save.bin'))[0][0]

CPU times: user 0 ns, sys: 952 ms, total: 952 ms
Wall time: 995 ms


In [5]:
dglG

Graph(num_nodes=170089, num_edges=53216794,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
for i in features.columns:
    dglG.ndata[i] = torch.tensor(features[i])

In [6]:
dglG

Graph(num_nodes=170089, num_edges=53216794,
      ndata_schemes={}
      edata_schemes={})